In [3]:
from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
from requests import Request, Session, Response

class FtxClient:


    def __init__(self, api_key=None, api_secret=None, subaccount_name=None):
            self._session = Session()
            self._api_key = api_key
            self._api_secret = api_secret
            self._subaccount_name = subaccount_name
            self._ENDPOINT = 'https://ftx.com/api/'


    def sign_request(self, request: Request):
        ts = int(time.time() * 1000)
        request = Request('GET', self._ENDPOINT)
        prepared = request.prepare()
        print(prepared)
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)

    def send_request(self, method: str, path: str, **kwargs):
        request = Request(method, self._ENDPOINT + path, **kwargs)
        self.sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _process_response(self, response: Response):
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']



In [9]:
acc = FtxClient(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)
acc.send_request('Get', '/futures/BTC-PERP')

<PreparedRequest [GET]>


{'ask': 57746.0,
 'bid': 57745.0,
 'change1h': 0.0033185648510120755,
 'change24h': 0.02060798868858254,
 'changeBod': 0.03485600616476407,
 'description': 'Bitcoin Perpetual Futures',
 'enabled': True,
 'expired': False,
 'expiry': None,
 'expiryDescription': 'Perpetual',
 'group': 'perpetual',
 'imfFactor': 0.002,
 'index': 57641.21987948,
 'last': 57761.0,
 'lowerBound': 54760.0,
 'marginPrice': 57746.0,
 'mark': 57746.0,
 'moveStart': None,
 'name': 'BTC-PERP',
 'perpetual': True,
 'positionLimitWeight': 1.0,
 'postOnly': False,
 'priceIncrement': 1.0,
 'sizeIncrement': 0.0001,
 'type': 'perpetual',
 'underlying': 'BTC',
 'underlyingDescription': 'Bitcoin',
 'upperBound': 60608.0,
 'volume': 80404.4076,
 'volumeUsd24h': 4528467605.3702}